# **RETO PERU** 
## *PRIORIZACION DE DESCUENTOS*
### Crear un modelo que estime la probabilidad de que un cliente tome una promoción y así encontrar las mejores promociones en los clientes con mayor probabilidad de adherirse a ellas.

EQUIPO: JicaScience_PE

In [ ]:
from google.colab import drive
drive.mount('/drive')                                                                                                                                                                                                                                                       

Mounted at /drive


In [ ]:
# Test the path
%ls /drive/MyDrive/Hackatones/Backus/data/input_data/

active_promos.csv  clients_attributes.csv  executed_promos.csv  sales.csv


## 1. Cargar la data 

In [ ]:
import pandas as pd
import datetime

In [ ]:
folder_input = "/drive/MyDrive/Hackatones/Backus/data/input_data/"
folder_output = "/drive/MyDrive/Hackatones/Backus/data/output_data/"
file_clients = folder_input + "clients_attributes.csv"  
file_promos_ex = folder_input + "executed_promos.csv"
file_promos_ac = folder_input + "active_promos.csv"
file_sales = folder_input + "sales.csv"

### - Crear el DataFrame **Atributos de los clientes** 

In [ ]:
df_clients = pd.read_csv(file_clients, parse_dates=["FechaAltaCliente"])

In [ ]:
df_clients.head()

,Cliente,FechaAltaCliente,Region,Gerencia,SubCanal,TipoPoblacion,Estrato,EF
0,1,2009-03-03,4,21,12,1,1,1
1,9,2009-03-03,4,21,9,2,3,1
2,15,2009-03-03,4,21,9,2,3,0
3,37,2009-03-03,4,21,16,2,3,1
4,43,2009-03-03,4,21,9,2,3,0


In [ ]:
df_clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15069 entries, 0 to 15068
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Cliente           15069 non-null  int64         
 1   FechaAltaCliente  15069 non-null  datetime64[ns]
 2   Region            15069 non-null  int64         
 3   Gerencia          15069 non-null  int64         
 4   SubCanal          15069 non-null  int64         
 5   TipoPoblacion     15069 non-null  int64         
 6   Estrato           15069 non-null  int64         
 7   EF                15069 non-null  int64         
dtypes: datetime64[ns](1), int64(7)
memory usage: 941.9 KB


### - Crear el DataFrame **Promociones ejecutadas** 




In [ ]:
df_promos_ex = pd.read_csv(file_promos_ex)
df_promos_ex['Ejecuto_Promo']=1

In [ ]:
df_promos_ex.head()

,CodigoDC,Cliente,Marca,Cupo,Ejecuto_Promo
0,297601,8410,29,9,1
1,297601,9337,29,9,1
2,309843,7404,40,16,1
3,309874,4017,40,16,1
4,309910,6316,40,16,1


In [ ]:
df_promos_ex.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27794 entries, 0 to 27793
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   CodigoDC       27794 non-null  int64
 1   Cliente        27794 non-null  int64
 2   Marca          27794 non-null  int64
 3   Cupo           27794 non-null  int64
 4   Ejecuto_Promo  27794 non-null  int64
dtypes: int64(5)
memory usage: 1.1 MB


### - Crera el DataFrame **Promociones actuales** 

In [ ]:
df_promos_ac = pd.read_csv(file_promos_ac, parse_dates=["Fecha_Desde", "Fecha_Hasta"])

In [ ]:
df_promos_ac.head()

,CodigoDC,Marca,Cupo,Fecha_Desde,Fecha_Hasta,Cliente
0,297601,29,9,2018-08-06,2018-08-30,3213
1,297601,29,9,2018-08-06,2018-08-30,3795
2,297601,29,9,2018-08-06,2018-08-30,11816
3,298856,40,16,2018-08-12,2018-08-19,8444
4,309916,40,16,2018-08-12,2018-08-19,8488


In [ ]:
df_promos_ac.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 422437 entries, 0 to 422436
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   CodigoDC     422437 non-null  int64         
 1   Marca        422437 non-null  int64         
 2   Cupo         422437 non-null  int64         
 3   Fecha_Desde  422437 non-null  datetime64[ns]
 4   Fecha_Hasta  422437 non-null  datetime64[ns]
 5   Cliente      422437 non-null  int64         
dtypes: datetime64[ns](2), int64(4)
memory usage: 19.3 MB


### - Crear el DataFrame **Ventas** 

In [ ]:
df_sales = pd.read_csv(file_sales, encoding="ISO-8859-1", engine="python")

In [ ]:
df_sales.head()

,Cliente,Año,Mes,ClaseEnvase,SegmentoPrecio,Marca,Cupo,Nr,Hl,Dcto
0,1,2018,1,2,2.0,44,10.0,6352.94,0.03,0.0
1,1,2018,1,2,4.0,18,16.0,99130.50,0.45,0.0
2,1,2018,6,2,1.0,2,6.0,11605.40,0.07,-2880.0
3,1,2018,7,1,2.0,37,34.0,1944.01,0.01,0.0
4,1,2018,8,2,1.0,2,16.0,16250.18,0.06,0.0


In [ ]:
# Change to datetime variable: [Año, Mes] -> FechaVenta
def change_to_datetime(row):
  year = row[0]
  month = row[1]
  fecha = str(year) + "/" + str(month) 
  return datetime.datetime.strptime(fecha, "%Y/%m")

df_sales.insert(1,"FechaVenta",df_sales[["Año","Mes"]].apply(change_to_datetime, axis=1))

del df_sales["Año"]
del df_sales["Mes"]

df_sales.head()

,Cliente,FechaVenta,ClaseEnvase,SegmentoPrecio,Marca,Cupo,Nr,Hl,Dcto
0,1,2018-01-01,2,2.0,44,10.0,6352.94,0.03,0.0
1,1,2018-01-01,2,4.0,18,16.0,99130.50,0.45,0.0
2,1,2018-06-01,2,1.0,2,6.0,11605.40,0.07,-2880.0
3,1,2018-07-01,1,2.0,37,34.0,1944.01,0.01,0.0
4,1,2018-08-01,2,1.0,2,16.0,16250.18,0.06,0.0


In [ ]:
df_sales.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1894252 entries, 0 to 1894251
Data columns (total 9 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   Cliente         1894252 non-null  int64         
 1   FechaVenta      1894252 non-null  datetime64[ns]
 2   ClaseEnvase     1894252 non-null  int64         
 3   SegmentoPrecio  1894247 non-null  float64       
 4   Marca           1894252 non-null  int64         
 5   Cupo            1894247 non-null  float64       
 6   Nr              1894252 non-null  float64       
 7   Hl              1894252 non-null  float64       
 8   Dcto            1894252 non-null  float64       
dtypes: datetime64[ns](1), float64(5), int64(3)
memory usage: 130.1 MB


## 2. Obtener nuevos dataframes

## - Formar el DataFrame **Promociones completas**



In [ ]:
# Se unen los dataframes df_promos_ac con df_promos_ex
result = pd.merge(df_promos_ac, df_promos_ex, how='left', on=['CodigoDC',	'Cliente'	,'Marca',	'Cupo'])
result = result.fillna(0) 

In [ ]:
result.head()

,CodigoDC,Marca,Cupo,Fecha_Desde,Fecha_Hasta,Cliente,Ejecuto_Promo
0,297601,29,9,2018-08-06,2018-08-30,3213,0.0
1,297601,29,9,2018-08-06,2018-08-30,3795,0.0
2,297601,29,9,2018-08-06,2018-08-30,11816,1.0
3,298856,40,16,2018-08-12,2018-08-19,8444,1.0
4,309916,40,16,2018-08-12,2018-08-19,8488,1.0


In [ ]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 422437 entries, 0 to 422436
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   CodigoDC       422437 non-null  int64         
 1   Marca          422437 non-null  int64         
 2   Cupo           422437 non-null  int64         
 3   Fecha_Desde    422437 non-null  datetime64[ns]
 4   Fecha_Hasta    422437 non-null  datetime64[ns]
 5   Cliente        422437 non-null  int64         
 6   Ejecuto_Promo  422437 non-null  float64       
dtypes: datetime64[ns](2), float64(1), int64(4)
memory usage: 25.8 MB


## - Formar el DataFrame **Ventas - clientes**

In [ ]:
# Se unen los dataframes df_sales y df_clients uno al lado del otro 
clienteventas = pd.merge(df_sales, df_clients, how='inner', on=['Cliente'])

In [ ]:
clienteventas.head()

,Cliente,FechaVenta,ClaseEnvase,SegmentoPrecio,Marca,Cupo,Nr,Hl,Dcto,FechaAltaCliente,Region,Gerencia,SubCanal,TipoPoblacion,Estrato,EF
0,1,2018-01-01,2,2.0,44,10.0,6352.94,0.03,0.0,2009-03-03,4,21,12,1,1,1
1,1,2018-01-01,2,4.0,18,16.0,99130.50,0.45,0.0,2009-03-03,4,21,12,1,1,1
2,1,2018-06-01,2,1.0,2,6.0,11605.40,0.07,-2880.0,2009-03-03,4,21,12,1,1,1
3,1,2018-07-01,1,2.0,37,34.0,1944.01,0.01,0.0,2009-03-03,4,21,12,1,1,1
4,1,2018-08-01,2,1.0,2,16.0,16250.18,0.06,0.0,2009-03-03,4,21,12,1,1,1


In [ ]:
clienteventas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1894252 entries, 0 to 1894251
Data columns (total 16 columns):
 #   Column            Dtype         
---  ------            -----         
 0   Cliente           int64         
 1   FechaVenta        datetime64[ns]
 2   ClaseEnvase       int64         
 3   SegmentoPrecio    float64       
 4   Marca             int64         
 5   Cupo              float64       
 6   Nr                float64       
 7   Hl                float64       
 8   Dcto              float64       
 9   FechaAltaCliente  datetime64[ns]
 10  Region            int64         
 11  Gerencia          int64         
 12  SubCanal          int64         
 13  TipoPoblacion     int64         
 14  Estrato           int64         
 15  EF                int64         
dtypes: datetime64[ns](2), float64(5), int64(9)
memory usage: 245.7 MB


Haciendo algo de analisis del dataframe **clienteventas** ...

In [ ]:
# Hallando la cantidad de compras que hace un cliente dado 
a=clienteventas[clienteventas['Cliente']==1]
print(f"El cliente 1 tuvo {len(a)} compras.")
a.describe()
print(f"Las marcas distintas que compró fueron: {a['Marca'].unique()}")


El cliente 1 tuvo 179 compras.
Las marcas distintas que compró fueron: [44 18  2 37  6  9  1 40 43 10 46 28 23 45 17 29 31]


In [ ]:
# Hallando la cantidad de promociones a las que accede el mismo cliente
b=result[result['Cliente']==1]
print(f"El cliente 1 accedió a {len(b)} promociones.")
b.describe()
print(f"Las marcas distintas a las que accedió con promoción fueron: {b['Marca'].unique()}")

El cliente 1 accedió a 39 promociones.
Las marcas distintas a las que accedió con promoción fueron: [40 39 29]


## - Formar el DataFrame **Dataset**
Este dataframe integra informacion de clientes, ventas y promociones

In [ ]:
# Unir los dfs: clienteventas y result, uno al lado del otro
Dataset = pd.merge(clienteventas, result, how='left', on=['Cliente'	,'Marca',	'Cupo'])

In [ ]:
Dataset.tail()

,Cliente,FechaVenta,ClaseEnvase,SegmentoPrecio,Marca,Cupo,Nr,Hl,Dcto,FechaAltaCliente,Region,Gerencia,SubCanal,TipoPoblacion,Estrato,EF,CodigoDC,Fecha_Desde,Fecha_Hasta,Ejecuto_Promo
4555207,15011,2019-07-01,1,2.0,44,29.0,8051.17,0.04,0.0,2019-02-22,5,19,9,2,2,0,NaN,NaT,NaT,NaN
4555208,15011,2019-03-01,2,1.0,6,16.0,11221.47,0.03,0.0,2019-02-22,5,19,9,2,2,0,NaN,NaT,NaT,NaN
4555209,15011,2019-03-01,2,1.0,40,16.0,9565.08,0.03,0.0,2019-02-22,5,19,9,2,2,0,NaN,NaT,NaT,NaN
4555210,15011,2019-04-01,1,2.0,44,29.0,7729.12,0.04,-323.7,2019-02-22,5,19,9,2,2,0,NaN,NaT,NaT,NaN
4555211,15011,2019-07-01,2,1.0,2,16.0,16451.46,0.06,0.0,2019-02-22,5,19,9,2,2,0,NaN,NaT,NaT,NaN


----- Calculando la antiguedad de un cliente

In [ ]:
# Calcular la antiguedad de un cliente, e insertarlo en la columna 16
Dataset.insert(16,"Antiguedad", Dataset["FechaVenta"] - Dataset["FechaAltaCliente"])
# List clients
clients_list = Dataset.groupby("Cliente")
aux = clients_list["Antiguedad"].max() # get the max value of each client
aux = aux.to_frame()
# Join dataframes
Dataset = Dataset.merge(aux, how='left', left_on="Cliente", right_index=True, suffixes=('_i','_c'))
# Copy, delete and rename
Dataset["Antiguedad_i"] = Dataset["Antiguedad_c"]
del Dataset["Antiguedad_c"]
Dataset.rename(columns={"Antiguedad_i":"Antiguedad"},inplace=True)

In [ ]:
Dataset.head()

,Cliente,FechaVenta,ClaseEnvase,SegmentoPrecio,Marca,Cupo,Nr,Hl,Dcto,FechaAltaCliente,Region,Gerencia,SubCanal,TipoPoblacion,Estrato,EF,Antiguedad,CodigoDC,Fecha_Desde,Fecha_Hasta,Ejecuto_Promo
0,1,2018-01-01,2,2.0,44,10.0,6352.94,0.03,0.0,2009-03-03,4,21,12,1,1,1,3803 days,NaN,NaT,NaT,NaN
1,1,2018-01-01,2,4.0,18,16.0,99130.50,0.45,0.0,2009-03-03,4,21,12,1,1,1,3803 days,NaN,NaT,NaT,NaN
2,1,2018-06-01,2,1.0,2,6.0,11605.40,0.07,-2880.0,2009-03-03,4,21,12,1,1,1,3803 days,NaN,NaT,NaT,NaN
3,1,2018-07-01,1,2.0,37,34.0,1944.01,0.01,0.0,2009-03-03,4,21,12,1,1,1,3803 days,NaN,NaT,NaT,NaN
4,1,2018-08-01,2,1.0,2,16.0,16250.18,0.06,0.0,2009-03-03,4,21,12,1,1,1,3803 days,NaN,NaT,NaT,NaN


In [ ]:
Dataset.describe()

,Cliente,ClaseEnvase,SegmentoPrecio,Marca,Cupo,Nr,Hl,Dcto,Region,Gerencia,SubCanal,TipoPoblacion,Estrato,EF,Antiguedad,CodigoDC,Ejecuto_Promo
count,4.555212e+06,4.555212e+06,4.555207e+06,4.555212e+06,4.555207e+06,4.555212e+06,4.555212e+06,4.555212e+06,4.555212e+06,4.555212e+06,4.555212e+06,4.555212e+06,4.555212e+06,4.555212e+06,4555212,2.810552e+06,2.810552e+06
mean,6.555949e+03,1.782569e+00,1.355469e+00,3.489898e+01,1.845227e+01,1.538326e+05,5.183978e-01,-9.356362e+03,2.944171e+00,1.664741e+01,1.231289e+01,1.764739e+00,2.242560e+00,5.335552e-01,2203 days 10:50:49.388385344,5.161113e+05,1.547241e-01
std,3.914006e+03,4.125012e-01,7.638901e-01,1.198155e+01,5.637584e+00,9.338870e+05,3.124392e+00,2.681667e+05,1.447078e+00,8.307449e+00,4.185103e+00,4.241619e-01,8.927077e-01,4.988728e-01,1193 days 04:06:33.218044592,1.063939e+05,3.616415e-01
min,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,-1.432168e+08,0.000000e+00,-1.630897e+08,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,127 days 00:00:00,2.976010e+05,0.000000e+00
25%,3.231000e+03,2.000000e+00,1.000000e+00,3.900000e+01,1.600000e+01,1.255168e+04,4.000000e-02,-2.112220e+03,1.000000e+00,9.000000e+00,9.000000e+00,2.000000e+00,2.000000e+00,0.000000e+00,1046 days 00:00:00,4.114340e+05,0.000000e+00
50%,6.224000e+03,2.000000e+00,1.000000e+00,4.000000e+01,1.600000e+01,3.797912e+04,1.200000e-01,0.000000e+00,3.000000e+00,1.600000e+01,1.200000e+01,2.000000e+00,2.000000e+00,1.000000e+00,2165 days 00:00:00,4.962510e+05,0.000000e+00
75%,9.758000e+03,2.000000e+00,1.000000e+00,4.000000e+01,2.000000e+01,1.147417e+05,3.900000e-01,0.000000e+00,4.000000e+00,2.400000e+01,1.500000e+01,2.000000e+00,3.000000e+00,1.000000e+00,3411 days 00:00:00,6.039970e+05,0.000000e+00
max,1.506900e+04,3.000000e+00,4.000000e+00,5.000000e+01,3.500000e+01,1.772279e+08,5.616200e+02,0.000000e+00,5.000000e+00,3.700000e+01,2.700000e+01,2.000000e+00,6.000000e+00,1.000000e+00,3803 days 00:00:00,6.884290e+05,1.000000e+00


In [ ]:
Dataset.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4555212 entries, 0 to 4555211
Data columns (total 21 columns):
 #   Column            Non-Null Count    Dtype          
---  ------            --------------    -----          
 0   Cliente           4555212 non-null  int64          
 1   FechaVenta        4555212 non-null  datetime64[ns] 
 2   ClaseEnvase       4555212 non-null  int64          
 3   SegmentoPrecio    4555207 non-null  float64        
 4   Marca             4555212 non-null  int64          
 5   Cupo              4555207 non-null  float64        
 6   Nr                4555212 non-null  float64        
 7   Hl                4555212 non-null  float64        
 8   Dcto              4555212 non-null  float64        
 9   FechaAltaCliente  4555212 non-null  datetime64[ns] 
 10  Region            4555212 non-null  int64          
 11  Gerencia          4555212 non-null  int64          
 12  SubCanal          4555212 non-null  int64          
 13  TipoPoblacion     4555212 n

**Algunas simplificaciones ...**

------ Eliminar todos los valores nulos

In [ ]:
# Eliminar todos los valores nulos
Dataset=Dataset.dropna()
Dataset.describe()
Dataset.head()

,Cliente,FechaVenta,ClaseEnvase,SegmentoPrecio,Marca,Cupo,Nr,Hl,Dcto,FechaAltaCliente,Region,Gerencia,SubCanal,TipoPoblacion,Estrato,EF,Antiguedad,CodigoDC,Fecha_Desde,Fecha_Hasta,Ejecuto_Promo
17,1,2019-02-01,2,1.0,40,16.0,38896.3,0.12,0.0,2009-03-03,4,21,12,1,1,1,3803 days,357832.0,2018-09-07,2018-09-11,0.0
18,1,2019-02-01,2,1.0,40,16.0,38896.3,0.12,0.0,2009-03-03,4,21,12,1,1,1,3803 days,355107.0,2018-09-05,2018-09-06,0.0
19,1,2019-02-01,2,1.0,40,16.0,38896.3,0.12,0.0,2009-03-03,4,21,12,1,1,1,3803 days,408714.0,2018-09-20,2018-09-29,0.0
20,1,2019-02-01,2,1.0,40,16.0,38896.3,0.12,0.0,2009-03-03,4,21,12,1,1,1,3803 days,405557.0,2018-09-17,2018-09-23,0.0
21,1,2019-02-01,2,1.0,40,16.0,38896.3,0.12,0.0,2009-03-03,4,21,12,1,1,1,3803 days,411125.0,2018-10-04,2018-10-04,0.0


In [ ]:
Dataset.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2810552 entries, 17 to 4555186
Data columns (total 21 columns):
 #   Column            Non-Null Count    Dtype          
---  ------            --------------    -----          
 0   Cliente           2810552 non-null  int64          
 1   FechaVenta        2810552 non-null  datetime64[ns] 
 2   ClaseEnvase       2810552 non-null  int64          
 3   SegmentoPrecio    2810552 non-null  float64        
 4   Marca             2810552 non-null  int64          
 5   Cupo              2810552 non-null  float64        
 6   Nr                2810552 non-null  float64        
 7   Hl                2810552 non-null  float64        
 8   Dcto              2810552 non-null  float64        
 9   FechaAltaCliente  2810552 non-null  datetime64[ns] 
 10  Region            2810552 non-null  int64          
 11  Gerencia          2810552 non-null  int64          
 12  SubCanal          2810552 non-null  int64          
 13  TipoPoblacion     2810552 

------ Filtrar o acotar datos

In [ ]:
# Filtrar los clientes
#Dataset=Dataset[Dataset['Cliente']<355]
#Dataset=Dataset.groupby(['Cliente','Marca','Cupo']).agg('mean').reset_index()
#Dataset=Dataset[Dataset['Mes']<2]
Dataset

,Cliente,FechaVenta,ClaseEnvase,SegmentoPrecio,Marca,Cupo,Nr,Hl,Dcto,FechaAltaCliente,Region,Gerencia,SubCanal,TipoPoblacion,Estrato,EF,Antiguedad,CodigoDC,Fecha_Desde,Fecha_Hasta,Ejecuto_Promo
17,1,2019-02-01,2,1.0,40,16.0,38896.30,0.12,0.0,2009-03-03,4,21,12,1,1,1,3803 days,357832.0,2018-09-07,2018-09-11,0.0
18,1,2019-02-01,2,1.0,40,16.0,38896.30,0.12,0.0,2009-03-03,4,21,12,1,1,1,3803 days,355107.0,2018-09-05,2018-09-06,0.0
19,1,2019-02-01,2,1.0,40,16.0,38896.30,0.12,0.0,2009-03-03,4,21,12,1,1,1,3803 days,408714.0,2018-09-20,2018-09-29,0.0
20,1,2019-02-01,2,1.0,40,16.0,38896.30,0.12,0.0,2009-03-03,4,21,12,1,1,1,3803 days,405557.0,2018-09-17,2018-09-23,0.0
21,1,2019-02-01,2,1.0,40,16.0,38896.30,0.12,0.0,2009-03-03,4,21,12,1,1,1,3803 days,411125.0,2018-10-04,2018-10-04,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4555182,15002,2019-07-01,2,1.0,40,16.0,136720.30,0.42,0.0,2019-02-21,3,9,16,2,1,0,161 days,683179.0,2019-09-22,2019-09-26,0.0
4555183,15002,2019-08-01,2,1.0,40,16.0,169882.78,0.53,-5930.7,2019-02-21,3,9,16,2,1,0,161 days,633339.0,2019-07-11,2019-07-30,0.0
4555184,15002,2019-08-01,2,1.0,40,16.0,169882.78,0.53,-5930.7,2019-02-21,3,9,16,2,1,0,161 days,641860.0,2019-08-07,2019-08-20,1.0
4555185,15002,2019-08-01,2,1.0,40,16.0,169882.78,0.53,-5930.7,2019-02-21,3,9,16,2,1,0,161 days,672200.0,2019-09-11,2019-09-17,0.0


## 3. Entrenar el modelo 
Se prueban varios algoritmos

In [ ]:
x = Dataset.iloc[:,[0,2,3,4,5,7,6,8,10,11,12,13,14,15,17]].values
y = Dataset.iloc[:,[20]].values.astype(float)
#y = y.to_numpy()

#3 Feature Scaling
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
#x = sc_X.fit_transform(x)

print(x)
print(y)

# Import train_test_split function
from sklearn.model_selection import train_test_split
x_train , x_test, y_train, y_test = train_test_split(x, y, train_size = 0.8, random_state = 40)

[[1.00000e+00 2.00000e+00 1.00000e+00 ... 1.00000e+00 1.00000e+00
  3.57832e+05]
 [1.00000e+00 2.00000e+00 1.00000e+00 ... 1.00000e+00 1.00000e+00
  3.55107e+05]
 [1.00000e+00 2.00000e+00 1.00000e+00 ... 1.00000e+00 1.00000e+00
  4.08714e+05]
 ...
 [1.50020e+04 2.00000e+00 1.00000e+00 ... 1.00000e+00 0.00000e+00
  6.41860e+05]
 [1.50020e+04 2.00000e+00 1.00000e+00 ... 1.00000e+00 0.00000e+00
  6.72200e+05]
 [1.50020e+04 2.00000e+00 1.00000e+00 ... 1.00000e+00 0.00000e+00
  6.83179e+05]]
[[0.]
 [0.]
 [0.]
 ...
 [1.]
 [0.]
 [0.]]


In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import numpy as np


import numpy as np
from sklearn.svm import SVR
import matplotlib.pyplot as plt

regressor = make_pipeline(StandardScaler(), SVR(kernel='poly'))
regressor.fit(x_train, y_train.ravel())

#regressor = SVR(kernel='rbf')#), C=100, gamma='auto', degree=4, epsilon=.1, coef0=1)
#regressor.fit(x_train, y_train.ravel())

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, regressor.predict(x_test))

In [ ]:
 from sklearn.metrics import r2_score

r2_score(y_test, regressor.predict(x_test))

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

regr = MLPRegressor(random_state=1, max_iter=500).fit(x_train, y_train.ravel())
#regr.predict(x_test[:2])



In [ ]:
regr.score(x_test, y_test)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.metrics import confusion_matrix
#from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz


sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

#Import svm model
from sklearn import svm

#Create a svm Classifier
clf = svm.SVC(kernel='rbf') # RAdial basis function Kernel

#Train the model using the training sets
clf.fit(x_train, y_train.ravel())

#Predict the response for test dataset
y_pred = clf.predict(x_test)

#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

model_2 = DecisionTreeClassifier()
model_2.fit(x_train, y_train.ravel())
predictions_DT = model_2.predict(x_test)

print('Decision tree accuracy:', accuracy_score(predictions_DT, y_test))
print('')
print('Confusion matrix:')
print(confusion_matrix(y_test,predictions_DT))

Decision tree accuracy: 0.9586095984600906

Confusion matrix:
[[462638  12232]
 [ 11034  76207]]


In [ ]:
predictions_DT

array([0., 0., 0., ..., 1., 0., 0.])

In [ ]:
df = pd.DataFrame(data=predictions_DT, columns=["Ejecuto_Promo"])
print(df)

        Ejecuto_Promo
0                 0.0
1                 0.0
2                 0.0
3                 0.0
4                 1.0
...               ...
562106            1.0
562107            0.0
562108            1.0
562109            0.0
562110            0.0

[562111 rows x 1 columns]


In [ ]:
df2 = pd.DataFrame(data=x_test)
df2=df2.iloc[:,[0,4,5]]
df2.columns=["Cliente","Marca","Cupo"]
print(df2)

        Cliente  Marca  Cupo
0       11478.0   16.0  0.15
1         328.0   16.0  0.68
2        9560.0   16.0  0.45
3        5815.0   16.0  0.15
4        7888.0   16.0  0.33
...         ...    ...   ...
562106  12191.0   16.0  0.06
562107   2415.0   20.0  0.03
562108   9292.0   16.0  0.03
562109  14196.0    9.0  0.05
562110   7434.0   16.0  0.09

[562111 rows x 3 columns]


In [ ]:
output=df2.join(df)
output

,Cliente,Marca,Cupo,Ejecuto_Promo
0,11478.0,16.0,0.15,0.0
1,328.0,16.0,0.68,0.0
2,9560.0,16.0,0.45,0.0
3,5815.0,16.0,0.15,0.0
4,7888.0,16.0,0.33,1.0
...,...,...,...,...
562106,12191.0,16.0,0.06,1.0
562107,2415.0,20.0,0.03,0.0
562108,9292.0,16.0,0.03,1.0
562109,14196.0,9.0,0.05,0.0


In [ ]:
df.to_csv(r'Path where you want to store the exported CSV file\File Name.csv', index = False)